### Test DualKuka in Comp vgrp Env

In [ ]:
%matplotlib inline
import time, sys
sys.path.append('..')
from pb_diff_envs.environment.static.comp_kuka_env import ComposedDynKukaEnv

import numpy as np
from importlib import reload
import matplotlib.pyplot as plt
import pybullet as p
import pb_diff_envs.objects.static.voxel_group as vv
reload(vv)
from pb_diff_envs.environment.static.kuka_rand_vgrp import Kuka_VoxelRandGroupList

mazelist_config = dict(planner_timeout=30, 
                        interp_density=1.5,
                       min_episode_distance=4*np.pi)

import gym
env_name = 'Compdualkuka14d-testOnly-rand-v12' 
# env_name = 'Compdualkuka14d-rand-nv5nv3-se1005-vr0702-hExt22-v0'
dvg_group = gym.make(env_name, load_mazeEnv=False, gen_data=True, is_eval=True) # True


## simple vis of env

In [ ]:
if p.isConnected():
    p.disconnect()
env = dvg_group.create_single_env(3) # 
env.unload_env()
env.load(GUI=False) # important not delete
p.getContactPoints()
p.stepSimulation()
plt.imshow(env.render())
plt.show()

p.configureDebugVisualizer(p.COV_ENABLE_GUI, 0)  # Disable the default GUI controls
p.configureDebugVisualizer(p.COV_ENABLE_MOUSE_PICKING, 1)  # Enable mouse picking for camera control
p.addUserDebugLine([0, 0, 0], [1, 0, 0], lineColorRGB=[1, 0, 0], lineWidth=6)
p.addUserDebugLine([0, 0, 0], [0, 1, 0], lineColorRGB=[0, 1, 0], lineWidth=6)
p.addUserDebugLine([0, 0, 0], [0, 0, 1], lineColorRGB=[0, 0, 1], lineWidth=6)
no_while = True ## can be True when with
while True:
    p.stepSimulation()
    # p.performCollisionDetection()
    c_pts = p.getContactPoints()
    # print(p.get)
    time.sleep(0.1)
    if c_pts is not None and len(c_pts) > 0:
        # print(p.getContactPoints())
        pass
        # time.sleep(2)
        # p.resetJointStatesMultiDof(env.robot_id, np.arange(7), [[0.]]*7)
    if no_while:
        break

In [ ]:
env.robot.print_joint_states()
## print(env.robot.item_id)
print('robot_id:', env.robot_id) # list
print('limits_high:', env.robot.limits_high)
# env.robot.limits_low
print(env._get_joint_pose())
print(f'{type(env._get_joint_pose())}')


## Sample a problem/episode

In [ ]:
from pb_diff_envs.utils.utils import DotDict
from tqdm import tqdm
env.load(GUI=False) # False
prev_pose = np.array([0.0] * env.robot.config_dim)
env.robot.set_config(prev_pose)
solutions = []
n_traj = 5 # 20
for i in tqdm(range(n_traj)):
    solution, _ = env.sample_1_episode(prev_pose)
    solution = np.array(solution)
    prev_pose = solution[-1]
    solutions.append(solution)

# solutions = np.concatenate(solutions, axis=0)
# result = DotDict(solution=solutions)
# print(solutions.shape)
# print(len(result.solution))
# result

In [ ]:
import matplotlib.pyplot as plt
import pybullet as p
import pybullet_data
import numpy as np
from pb_diff_envs.utils.utils import save_gif
from IPython.display import HTML
import base64
from pb_diff_envs.objects.trajectory import WaypointLinearTrajectory, WaypointProportionTrajectory
from pb_diff_envs.utils.kuka_utils_luo import SolutionInterp
# Visualization
from importlib import reload
import pb_diff_envs.utils.robogroup_utils_luo  as rul; reload(rul)
from pb_diff_envs.utils.robogroup_utils_luo import robogroup_visualize_traj_luo
from pb_diff_envs.utils.kuka_utils_luo import visualize_kuka_traj_luo


# pick one problem
sol_interp = SolutionInterp(density=1.5) # mazelist_config['interp_density']
sol_i = solutions[0] # which motion trajectory to visualize
result = DotDict(solution=sol_i)
print(result.solution.shape)
traj_vis = sol_interp(result.solution,)

gifs, ds, vis_dict = robogroup_visualize_traj_luo(env, traj_vis, is_debug=False)


gif_name = './test_comp_dualkuka_wgrp.gif'
save_gif(gifs, gif_name, duration=ds)
b64 = base64.b64encode(open(gif_name, 'rb').read()).decode('ascii')
display(HTML(f'<img src="data:image/gif;base64,{b64}" />'))

print(traj_vis.shape)
print(result.solution.shape)

In [ ]:
env.wall_locations
# while True:
#     time.sleep(0.1)
#     p.stepSimulation()